In [167]:
#coding:utf-8
import pylab
from PIL import Image,ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math
import copy

#画像の読み込み
test = cv2.imread("./Img/input.jpg", cv2.IMREAD_COLOR)#BGRなので気をつける
gray_test = cv2.imread("./Img/input.jpg",cv2.IMREAD_GRAYSCALE)
width = test.shape[1]
height = test.shape[0] 

#画像の書き出し
cv2.imwrite('./Img/test.jpg', test)
cv2.imwrite('./Img/gray_test.jpg',gray_test)



True

In [168]:
#学習用テストケース
shape = np.zeros((24,2))
shape[:,0] = [12,97,24,62,56,42,96,30,125,34,151,48,178,94,174,137,136,158,100,165,62,158,32,136]
shape[:,1] = [49,173,27,143,24,105,46,57,78,31,114,23,150,36,175,73,172,113,150,148,120,172,85,185]

# shape = np.zeros((4,2))
# shape[:,0] = [1,1,1,2]
# shape[:,1] = [3,3,3,4]
shape

array([[  12.,   49.],
       [  97.,  173.],
       [  24.,   27.],
       [  62.,  143.],
       [  56.,   24.],
       [  42.,  105.],
       [  96.,   46.],
       [  30.,   57.],
       [ 125.,   78.],
       [  34.,   31.],
       [ 151.,  114.],
       [  48.,   23.],
       [ 178.,  150.],
       [  94.,   36.],
       [ 174.,  175.],
       [ 137.,   73.],
       [ 136.,  172.],
       [ 158.,  113.],
       [ 100.,  150.],
       [ 165.,  148.],
       [  62.,  120.],
       [ 158.,  172.],
       [  32.,   85.],
       [ 136.,  185.]])

In [169]:
#テスト用
black = np.zeros((200,200))
N = int(len(shape[:,0])/2)
for i in range(0,int(N)):
    x = int(shape[2*i][0])
    y = int(shape[2*i+1][0])
    x2 = int(shape[(2*(i+1))%(2*N)][0])
    y2 = int(shape[(2*(i+1)+1)%(2*N)][0])
    black = cv2.line(black, (y,x), (y2,x2), (255, 255, 255), 2)
cv2.imwrite('./Img/black.jpg' ,black)

black = np.zeros((200,200))
N = int(len(shape[:,1])/2)
for i in range(0,int(N)):
    x = int(shape[2*i][1])
    y = int(shape[2*i+1][1])
    x2 = int(shape[(2*i+2)%(2*N)][1])
    y2 = int(shape[(2*i+3)%(2*N)][1])
    black = cv2.line(black, (y,x), (y2,x2), (255, 255, 255), 2)
cv2.imwrite('./Img/black2.jpg' ,black)

True

In [170]:
black = np.zeros((200,200))
shape_ave = (shape[:,0]+shape[:,1])/2
for i in range(0,int(N)):
    x = int(shape_ave[2*i])
    y = int(shape_ave[2*i+1])
    x2 = int(shape_ave[(2*(i+1))%(2*N)])
    y2 = int(shape_ave[(2*(i+1)+1)%(2*N)])
    black = cv2.line(black, (y,x), (y2,x2), (255, 255, 255), 2)
cv2.imwrite('./Img/blackave.jpg' ,black)
print(shape_ave)

[  30.5  135.    25.5  102.5   40.    73.5   71.    43.5  101.5   32.5
  132.5   35.5  164.    65.   174.5  105.   154.   135.5  125.   156.5
   91.   165.    58.5  160.5]


In [183]:
sample = shape.shape[1]
S = np.zeros((2*N,2*N))
for i in range(0,2*N):
    for j in range(0,2*N):
        for n in range(0,sample):
            S[i,j] += (shape[i][n]-shape_ave[i])*(shape[j][n]-shape_ave[j])

S /= sample-1
print(S)
# ma = np.max(S)
# S /= ma
# S *= 255
# cv2.imwrite('./Img/s.jpg' ,S)

[[  6.84500000e+02   1.40600000e+03   5.55000000e+01   1.49850000e+03
   -5.92000000e+02   1.16550000e+03  -9.25000000e+02   4.99500000e+02
   -8.69500000e+02  -5.55000000e+01  -6.84500000e+02  -4.62500000e+02
   -5.18000000e+02  -1.07300000e+03   1.85000000e+01  -1.18400000e+03
    6.66000000e+02  -8.32500000e+02   9.25000000e+02  -3.14500000e+02
    1.07300000e+03   2.59000000e+02   9.80500000e+02   9.06500000e+02]
 [  1.40600000e+03   2.88800000e+03   1.14000000e+02   3.07800000e+03
   -1.21600000e+03   2.39400000e+03  -1.90000000e+03   1.02600000e+03
   -1.78600000e+03  -1.14000000e+02  -1.40600000e+03  -9.50000000e+02
   -1.06400000e+03  -2.20400000e+03   3.80000000e+01  -2.43200000e+03
    1.36800000e+03  -1.71000000e+03   1.90000000e+03  -6.46000000e+02
    2.20400000e+03   5.32000000e+02   2.01400000e+03   1.86200000e+03]
 [  5.55000000e+01   1.14000000e+02   4.50000000e+00   1.21500000e+02
   -4.80000000e+01   9.45000000e+01  -7.50000000e+01   4.05000000e+01
   -7.05000000e+01

True

In [184]:
la, v = np.linalg.eigh(S)
print(u"固有値："+str(la))
print(u"固有ベクトル："+str(v))
# la[la<10**(-20)] = 0 #小さすぎる値は0 として近似
print(la) #最終的な固有値

固有値：[ -2.77321913e-13  -1.05383145e-13  -3.97481719e-14  -3.24866273e-14
  -1.21541419e-14  -8.22823858e-15  -5.32895485e-15  -2.27861771e-15
  -1.82648470e-15  -3.62377829e-16  -2.55680136e-16  -4.04576585e-18
  -1.27881748e-31   3.55289904e-17   3.73097134e-16   1.45722149e-15
   3.65668895e-15   5.87113260e-15   7.70050459e-15   1.98394856e-14
   2.97155500e-14   4.52535749e-14   2.35885580e-13   1.91508459e+03]
固有ベクトル：[[ -8.40611905e-01   4.96576638e-01  -4.95639390e-02   4.03534696e-02
    5.71016424e-02  -1.41845284e-02  -2.64378315e-02  -1.89199366e-02
   -6.68320687e-03  -1.56454836e-03   3.04247262e-03   5.23760759e-03
   -3.48238337e-16  -3.13928370e-03  -4.04398558e-03   1.58087223e-02
   -1.91514902e-02  -2.98177086e-02  -4.19454831e-02   4.14930313e-02
   -3.01880703e-02  -1.18964083e-02  -6.51644341e-02   1.66683578e-01]
 [  3.78974571e-01   4.02189228e-01  -6.67574129e-02   1.35661542e-01
    1.26157042e-02   3.99084931e-03  -1.47969537e-02  -3.32014285e-02
   -1.5091085

In [185]:
eigen_id = np.argsort(la)[::-1]
la = la[eigen_id]
v = v[:,eigen_id]
print(la)
print(v)

[  1.91508459e+03   2.35885580e-13   4.52535749e-14   2.97155500e-14
   1.98394856e-14   7.70050459e-15   5.87113260e-15   3.65668895e-15
   1.45722149e-15   3.73097134e-16   3.55289904e-17  -1.27881748e-31
  -4.04576585e-18  -2.55680136e-16  -3.62377829e-16  -1.82648470e-15
  -2.27861771e-15  -5.32895485e-15  -8.22823858e-15  -1.21541419e-14
  -3.24866273e-14  -3.97481719e-14  -1.05383145e-13  -2.77321913e-13]
[[  1.66683578e-01  -6.51644341e-02  -1.18964083e-02  -3.01880703e-02
    4.14930313e-02  -4.19454831e-02  -2.98177086e-02  -1.91514902e-02
    1.58087223e-02  -4.04398558e-03  -3.13928370e-03  -3.48238337e-16
    5.23760759e-03   3.04247262e-03  -1.56454836e-03  -6.68320687e-03
   -1.89199366e-02  -2.64378315e-02  -1.41845284e-02   5.71016424e-02
    4.03534696e-02  -4.95639390e-02   4.96576638e-01  -8.40611905e-01]
 [  3.42377079e-01  -7.36674210e-01  -2.23789078e-02   3.55549747e-02
    8.74872185e-02  -2.17368819e-03   8.49077795e-03  -1.36102881e-02
    1.19090678e-02  -6.6

In [186]:
la_sum = np.sum(la)
need_la = 0 #必要固有値の和
need_la_num = 1 #必要固有値数-1
for val in la:
    need_la += val
    if(la_sum*0.98 < need_la):
        break
    else:
        need_la_num +=1
print(need_la_num)

1


In [187]:
P = np.empty((N*2,need_la_num))
for j in range(0,need_la_num):
    P[:,j] = v[:,j]
b = np.empty((need_la_num))
print(P)
print(b)

[[ 0.16668358]
 [ 0.34237708]
 [ 0.01351488]
 [ 0.36490189]
 [-0.14415877]
 [ 0.28381258]
 [-0.22524808]
 [ 0.12163396]
 [-0.21173319]
 [-0.01351488]
 [-0.16668358]
 [-0.11262404]
 [-0.12613892]
 [-0.26128777]
 [ 0.00450496]
 [-0.28831754]
 [ 0.16217862]
 [-0.20272327]
 [ 0.22524808]
 [-0.07658435]
 [ 0.26128777]
 [ 0.06306946]
 [ 0.23876296]
 [ 0.22074312]]
[ 1915.08459076]


In [193]:
param=2#(±3までらしいが、、、)
for i in range(0,len(b)):
    b[i] = la[i]**(1/2)*param
# print(P.shape,b.shape,result.shape)
# print(P)
# print(b)
result = shape_ave+np.dot(P,b)

# print(result)
# 輪郭線のテスト
black_result = np.zeros((200,200))
N = int(len(shape[:,0])/2)
for i in range(0,int(N)):
    x = int(result[2*i])
    y = int(result[2*i+1])
    x2 = int(result[(2*(i+1))%(2*N)])
    y2 = int(result[(2*(i+1)+1)%(2*N)])
    print(x,y,x2,y2)
    black_result = cv2.line(black_result, (y,x), (y2,x2), (255, 255, 255), 2)
cv2.imwrite('./Img/black_result.jpg' ,black_result)

45 164 26 134
26 134 27 98
27 98 51 54
51 54 82 31
82 31 117 25
117 25 152 42
152 42 174 79
174 79 168 117
168 117 144 149
144 149 113 170
113 170 79 179
79 179 45 164


True

In [368]:
#test
hoge= np.zeros((4,2))
fuga_  = np.zeros((4,2))
fuga_[:,0] =  [1,1,1,2]
fuga_[:,1] = [3,3,4,4]
hoge[:,0] = copy.deepcopy(fuga_[:,0])
hoge[:,1] = copy.deepcopy(fuga_[:,1])
print("移動前:\n",hoge)
#重心とる
N = int(len(hoge[:,0])/2)
g1_ = np.array([0,0])
g1_ = np.sum(hoge[::2,0])/N,np.sum(hoge[1::2,0])/N
g2_ = np.array([0,0])
g2_ = np.sum(hoge[::2,1])/N,np.sum(hoge[1::2,1])/N
hoge[::2,0] -= g1_[0]
hoge[1::2,0] -= g1_[1]
hoge[::2,1] -= g2_[0]
hoge[1::2,1] -= g2_[1]
print(g2_)
print("移動後:\n",hoge)
#パラメータ計算
a_ = np.dot(hoge[:,0],hoge[:,1])/np.dot(hoge[:,0],hoge[:,0])
b_ =0
loop = int(hoge[:,0].shape[0]/2)
for i in range(0,loop):
    x1_ = hoge[2*i,0]
    y1_ = hoge[2*i+1,0]
    x2_ = hoge[2*i,1]
    y2_ = hoge[2*i+1,1]
    b_ += (x1_*y2_-x2_*y1_ )
b_ /= np.dot(hoge[:,0],hoge[:,0])
s_ = (a_**2+b_**2)**(1/2)
theta_ = np.arctan2(b_,a_)
print("s:",s_,"θ:",theta_)

移動前:
 [[ 1.  3.]
 [ 1.  3.]
 [ 1.  4.]
 [ 2.  4.]]
(3.5, 3.5)
移動後:
 [[ 0.  -0.5]
 [-0.5 -0.5]
 [ 0.   0.5]
 [ 0.5  0.5]]
s: 1.41421356237 θ: -0.785398163397


In [373]:
sA_ =  np.zeros((2,2))
sA_[:,0] = [s_*np.cos(theta_),-s_*np.sin(theta_)]
sA_[:,1] = [s_*np.sin(theta_), s_*np.cos(theta_)]

ans_ = [0,0,0,0]
for i in range(0,loop):
    aho = [hoge[2*i,0],hoge[2*i+1,0]]
    ans_[2*i],ans_[2*i+1] = np.dot(aho,sA_) +g2_




In [374]:
ans_,fuga_[:,1]

([3.0, 3.0, 4.0, 4.0], array([ 3.,  3.,  4.,  4.]))